In [1]:
# For easier dev of local modules:
%load_ext autoreload
%autoreload 2

# Python Built-Ins:
import json

# External Dependencies:
import boto3
import sagemaker
from sagemaker.pytorch.estimator import PyTorch as PyTorchEstimator


In [2]:
BUCKET_NAME = "2020-05-gym-bert"
%store BUCKET_NAME

SQUAD_V2 = False  # Whether to use V2 (including unanswerable questions)
%store SQUAD_V2

Stored 'BUCKET_NAME' (str)
Stored 'SQUAD_V2' (bool)


In [3]:
role = sagemaker.get_execution_role()
botosess = boto3.session.Session()
region = botosess.region_name
s3 = botosess.resource("s3")
bucket = s3.Bucket(BUCKET_NAME)
smclient = botosess.client("sagemaker")

In [4]:
!mkdir -p models

In [5]:
bucket.Object(
    "bert-calssification-distributed-2020-05-05-15-58-03-622/output/output.tar.gz"
).download_file(
    "models/bert-cls.tar.gz"
)
bucket.Object(
    "distilbert-calssification-distributed-2020-05-05-16-24-55-728/output/output.tar.gz"
).download_file(
    "models/distilbert-cls.tar.gz"
)

In [ ]:
!mkdir -p models/bert-cls
!tar -C models/bert-cls -zxvf models/bert-cls.tar.gz
!mkdir -p models/distilbert-cls
!tar -C models/distilbert-cls -zxvf models/distilbert-cls.tar.gz

In [ ]:
!./install-sam.sh

In [3]:
STAGING_BUCKET = "2020-05-gym-bert-sam-staging"
STACK_NAME = "testa"
!./deploy.sh {STAGING_BUCKET} {STACK_NAME}

In [ ]:
#from transformers import pipeline,TextClassificationPipeline, AutoModelForTokenClassification, AutoTokenizer

# Sentiment analysis pipeline
#pipeline('sentiment-analysis')
#pipe=pipeline('sentiment-analysis',model=model,tokenizer=tokenizer)
#pipe=TextClassificationPipeline(model=model,tokenizer=tokenizer)

# Question answering pipeline, specifying the checkpoint identifier
#pipeline('question-answering', model='distilbert-base-cased-distilled-squad', tokenizer='bert-base-cased')

# Named entity recognition pipeline, passing in a specific model and tokenizer
#model = AutoModelForTokenClassification.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english")
#tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
#pipeline('ner', model=model, tokenizer=tokenizer)